In [1]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode
        
        if self.mode == 'train':
            self.classes = sorted(self.data['city'].unique())
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx]['filename'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.mode == 'train':
            city = self.data.iloc[idx]['city']
            label = self.class_to_idx[city]
            return image, label
        else:
            return image, self.data.iloc[idx]['filename']


In [2]:
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
train_csv = '/kaggle/input/datathon-ai-qualification-round/train_data.csv'
test_csv = '/kaggle/input/datathon-ai-qualification-round/test.csv'
train_dir = '/kaggle/input/datathon-ai-qualification-round/train/train'  # Adjust this to the correct path
test_dir = '/kaggle/input/datathon-ai-qualification-round/test/test'


# Split training CSV into train and validation sets
df = pd.read_csv(train_csv)
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['city'], random_state=42)

# Save temporary CSVs for train and val to load with dataset class
train_split_csv = "train_split.csv"
val_split_csv = "val_split.csv"
train_df.to_csv(train_split_csv, index=False)
val_df.to_csv(val_split_csv, index=False)

In [4]:

# Transform definitions
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [5]:
train_dataset = CustomDataset(csv_file=train_split_csv, root_dir=train_dir, transform=train_transforms, mode='train')
val_dataset = CustomDataset(csv_file=val_split_csv, root_dir=train_dir, transform=val_transforms, mode='train')
test_dataset = CustomDataset(csv_file=test_csv, root_dir=test_dir, transform=test_transforms, mode='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(train_dataset.class_to_idx)

# Load the EfficientNet-B0 model with pretrained weights
model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)

# Modify the classification head to match the number of classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 160MB/s]


In [7]:
epochs = 8
for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)
            _, val_preds = torch.max(outputs, 1)
            val_correct += (val_preds == labels).sum().item()
            val_total += labels.size(0)
    
    val_epoch_loss = val_running_loss / val_total
    val_epoch_acc = val_correct / val_total
    
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc*100:.2f}% | "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc*100:.2f}%")


# Save model
torch.save(model.state_dict(), "model30.pth")


Training Epoch 1/8: 100%|██████████| 186/186 [03:42<00:00,  1.20s/it]


Epoch [1/8], Train Loss: 0.5364, Train Acc: 78.72% | Val Loss: 0.3559, Val Acc: 86.29%


Training Epoch 2/8: 100%|██████████| 186/186 [03:27<00:00,  1.11s/it]


Epoch [2/8], Train Loss: 0.3289, Train Acc: 87.65% | Val Loss: 0.2551, Val Acc: 91.14%


Training Epoch 3/8: 100%|██████████| 186/186 [03:26<00:00,  1.11s/it]


...

Epoch [6/8], Train Loss: 0.1402, Train Acc: 95.08% | Val Loss: 0.3093, Val Acc: 89.33%


Training Epoch 7/8: 100%|██████████| 186/186 [03:24<00:00,  1.10s/it]


Epoch [7/8], Train Loss: 0.1273, Train Acc: 95.65% | Val Loss: 0.2609, Val Acc: 91.52%


Training Epoch 8/8: 100%|██████████| 186/186 [03:25<00:00,  1.11s/it]


Epoch [8/8], Train Loss: 0.1168, Train Acc: 95.70% | Val Loss: 0.2028, Val Acc: 93.52%


In [8]:

# Load model for inference (if needed)
model.load_state_dict(torch.load("model30.pth"))

# Inference on Test
model.eval()
predictions = []

idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        for f, p in zip(file_names, preds):
            city_name = idx_to_class[p.item()]
            predictions.append((f, city_name))

# Save predictions
pred_df = pd.DataFrame(predictions, columns=["filename", "city"])
pred_df.to_csv("test_predictionsefficiencynet30.csv", index=False)
print("Predictions saved to test_predictionsefficiencynet30.csv")



/tmp/ipykernel_23/1309391058.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model30.pth"))
Predicting on Test Set: 100%|██████████| 63

Predictions saved to test_predictionsefficiencynet30.csv
